In [3]:
import os, cv2, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPool2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils

/home/mzl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
TRAIN_DIR = 'train/'
TEST_DIR = 'test/'
ROWS = 128
COLS = 128
CHANNELS = 3
print(TRAIN_DIR)

train/


In [19]:
# 加入全部数据
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]
train_dogs =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'dog' in i]
train_cats =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'cat' in i]

test_images =  [TEST_DIR+i for i in os.listdir(TEST_DIR)]
print(len(train_dogs),len(train_cats))

12500 12500


In [20]:
# 使用部分数据 
#train_images = train_dogs[:20000] + train_cats[:2000]
#test_images =  test_images[:25]
random.shuffle(train_images)

def read_image(file_path):
    img = cv2.imread(str(file_path), cv2.IMREAD_COLOR) #cv2.IMREAD_GRAYSCALE
    return cv2.resize(img,(ROWS, COLS),interpolation=cv2.INTER_CUBIC)


In [21]:
def prep_data(images):
    count = len(images)
    data = np.ndarray((count, ROWS, COLS,CHANNELS), dtype=np.uint8)

    for i,image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image
        if i%250 == 0: print('Processed {} of {}'.format(i, count))
    
    return data

In [22]:

train = prep_data(train_images)
test = prep_data(test_images)
print(train.shape)
print("Train shape: {}".format(train.shape))
print("Test shape: {}".format(test.shape))

Processed 0 of 25000
Processed 250 of 25000
Processed 500 of 25000
Processed 750 of 25000
Processed 1000 of 25000
Processed 1250 of 25000
Processed 1500 of 25000
Processed 1750 of 25000
Processed 2000 of 25000
Processed 2250 of 25000
Processed 2500 of 25000
Processed 2750 of 25000
Processed 3000 of 25000
Processed 3250 of 25000
Processed 3500 of 25000
Processed 3750 of 25000
Processed 4000 of 25000
Processed 4250 of 25000
Processed 4500 of 25000
Processed 4750 of 25000
Processed 5000 of 25000
Processed 5250 of 25000
Processed 5500 of 25000
Processed 5750 of 25000
Processed 6000 of 25000
Processed 6250 of 25000
Processed 6500 of 25000
Processed 6750 of 25000
Processed 7000 of 25000
Processed 7250 of 25000
Processed 7500 of 25000
Processed 7750 of 25000
Processed 8000 of 25000
Processed 8250 of 25000
Processed 8500 of 25000
Processed 8750 of 25000
Processed 9000 of 25000
Processed 9250 of 25000
Processed 9500 of 25000
Processed 9750 of 25000
Processed 10000 of 25000
Processed 10250 of 25

In [23]:
labels = []
for i in train_images:
    if 'dog' in i:
        labels.append(1)
    else:
        labels.append(0)
print(len(labels))

25000


In [24]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fscore(y_true, y_pred):
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    f_score = 2 * (p * r) / (p + r + K.epsilon())
    return f_score

In [36]:
from keras import backend as K
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'


def catdog():
    
    model = Sequential()
    
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (128,128,3)))
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',  
                 activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy',fscore])
    return model


model = catdog()

In [34]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto') 

In [37]:
from keras.callbacks import ModelCheckpoint
# 保存本次模型
model_checkpoint = ModelCheckpoint('weights9.h5', monitor='val_loss', save_best_only=True)
epochs = 20
batch_size = 64 # GPU 不好，不宜过高

history = LossHistory()


predictions = model.fit(train, labels, batch_size = batch_size, epochs = epochs, 
          validation_split = 0.1, verbose = 1,shuffle=False,callbacks=[history, early_stopping])

Train on 22500 samples, validate on 2500 samples
Epoch 1/20
22500/22500 [==============================] - 373s 17ms/step - loss: 0.7307 - acc: 0.6029 - fscore: 0.5756 - val_loss: 0.5414 - val_acc: 0.7360 - val_fscore: 0.7502
Epoch 2/20
22500/22500 [==============================] - 369s 16ms/step - loss: 0.5435 - acc: 0.7242 - fscore: 0.7139 - val_loss: 0.4757 - val_acc: 0.7820 - val_fscore: 0.7745
Epoch 3/20
22500/22500 [==============================] - 366s 16ms/step - loss: 0.4783 - acc: 0.7728 - fscore: 0.7675 - val_loss: 0.4337 - val_acc: 0.8036 - val_fscore: 0.8058
Epoch 4/20
22500/22500 [==============================] - 364s 16ms/step - loss: 0.4151 - acc: 0.8156 - fscore: 0.8131 - val_loss: 0.4096 - val_acc: 0.8156 - val_fscore: 0.8181
Epoch 5/20
22500/22500 [==============================] - 378s 17ms/step - loss: 0.3607 - acc: 0.8414 - fscore: 0.8398 - val_loss: 0.3820 - val_acc: 0.8340 - val_fscore: 0.8308
Epoch 6/20
22500/22500 [==============================] - 371s 16m

In [30]:
# 导入模型
#model.load_weights('weight.h5')
y_pred = model.predict(test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", (224, 224), shuffle=False, 
                                         batch_size=64, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)
df.head(20)

12500/12500 [==============================] - 66s 5ms/step
Found 12500 images belonging to 1 classes.


/home/mzl/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,id,label
0,1,0.155220
1,2,0.995000
2,3,0.109511
3,4,0.017239
4,5,0.005000
5,6,0.962003
6,7,0.995000
7,8,0.023719
8,9,0.988997
9,10,0.995000
